In [4]:
#Import API
from dora.api import DataExplorer

import pandas as pd
from datetime import date, timedelta, datetime
from matplotlib import pyplot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
import numpy as np

import random
import time
from sklearn.model_selection import KFold
import json
import sys

explorer = DataExplorer()

In [5]:
def clusterDist(df):
    maxn=df['cluster'].max()
    clusterStats=pd.DataFrame()
    for i in range(maxn+1):
        mask=(df['cluster']==i)
        t=df.loc[mask]
        clusterStats.loc[i,'avgNumOrders']=t['numorders'].mean()
        clusterStats.loc[i,'avgTotalSpent']=t['totalspent'].mean()
        clusterStats.loc[i,'numHouseholds']=t['householdid'].nunique()
        clusterStats.loc[i,'numZipcoes']=t['zipcode'].nunique()
        clusterStats.loc[i,'avgTotalPopMales']=t['totalmales'].mean()
        clusterStats.loc[i,'avgTotalPopFemales']=t['totalfemales'].mean()
        clusterStats.loc[i,'avgTotalPop']=t['totalpop'].mean()
        clusterStats.loc[i,'medianAge']=t['medianage'].median()
    return clusterStats

In [6]:
cluster1CustmeridsMapping = np.load('cluster1CustmeridsMapping.npy')

#cCluster_rmHID.columns

array(['gender', 'totalmales', 'totalfemales', 'totalpop',
       'customermatchedid', 'numorders', 'zipcode', 'medianage',
       'totalspent', 'householdid', 'firstname', 'numcustomerid',
       'cluster', 'customerids'], dtype=object)

In [7]:
# Create co-occurrence matrix

def create_cocmatrix(subset_matrix_cust_order):
    rows, cols = subset_matrix_cust_order.shape
    m = np.zeros((cols,cols))
    
    for i in range(cols):
        t = np.sum(subset_matrix_cust_order[subset_matrix_cust_order[:,i] > 0],axis=0)
        t[i] = 0
        m[i,:] = t
                    
    return m

In [8]:
def create_seed(rand):
    np.random.seed(rand) # create seed for repeatable results

In [9]:
# generate recommendations
# purchase_vec = customer purchases (vector)
# cocm = co occurrence matrix [items x items]
# num_rec = number of recommendations
def gen_recom(purchase_list,cocm,num_rec):
    rowsum = np.zeros(cocm.shape[0])
    
    for p in purchase_list:
        rowsum += cocm[p,:]
        
    rowsum[purchase_list] = 0
    indices = np.nonzero(rowsum)[0]
    toprec = indices[np.argsort(rowsum[indices])][-1 * num_rec:][::-1]
    return toprec

In [10]:
def get_indices(dataset, col, var=None):
    if var is None:
        return [y for y, x in enumerate(dataset[:,col])] # all
    
    return [y for y, x in enumerate(dataset[:,col]) if x == var]

In [11]:
def check_idx_size(idxlist,maximum_rows):
    if len(idxlist) > maximum_rows:
        return np.random.choice(idxlist, maximum_rows, replace=False)
    
    return idxlist

In [12]:
# customer to order lookup
matrix_co = np.load('cust_item_matrix.npy')
print ("created customer to order lookup")

matrix_co.shape

created customer to order lookup


(189559, 3990)

In [13]:
custlist = np.load('custids.npy')
print ("created customer id lookup")

custlist.shape

created customer id lookup


(189559,)

In [38]:
cluster1CustmeridsMapping[:5]

array([[ 5.],
       [ 6.],
       [ 7.],
       [ 0.],
       [ 4.]])

In [39]:
custlist[:5]

array([1, 2, 3, 4, 5])

In [75]:
unique_clusters = sorted(list(set(cluster1CustmeridsMapping[:,0])))
print (unique_clusters)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]


In [49]:
# bring in customerids to map with cluster ids
arr_cxd = np.zeros((custlist.shape[0],1))

In [50]:
arr_cxd.shape

(189559, 1)

In [44]:
#x = [[1,2,3],[4,5,6],[7,8,9]]
#x = np.delete(x, [0,1], axis=0)
#x

In [18]:
cluster1CustmeridsMapping.shape

(189559, 1)

In [60]:
# mapping cluster ids
for i in range(cluster1CustmeridsMapping.shape[0]):
    cid = cluster1CustmeridsMapping[i]
    arr_cxd[i] = cid    

In [32]:
#np.save('cluster1CustmeridsMapping.npy',arr_cxd)

In [85]:
# mapping cluster ids
def create_cluster_coo_matrix(mco,clusterid):
    global cluster1CustmeridsMapping
    indx = []
    
    for i in range(cluster1CustmeridsMapping.shape[0]):
        if (cluster1CustmeridsMapping[i]) == clusterid:
            indx.append(i)

    cluster_coo_matrix = create_cocmatrix(mco[indx])
    np.save("cluster{0}_coo_matrix.npy".format(clusterid),cluster_coo_matrix)

In [91]:
create_cluster_coo_matrix(matrix_co,7)

In [88]:
testm = np.load('cluster3_coo_matrix.npy')

In [89]:
testm.shape

(3990, 3990)

In [34]:
testm5 = np.load('cluster5_coo_matrix.npy')
testm5.shape

(3990, 3990)

In [36]:
testm6 = np.load('cluster6_coo_matrix.npy')
testm6.shape

(3990, 3990)

In [41]:
testm6[:5]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [61]:
cluster1CustmeridsMapping.shape

(189559, 1)

In [62]:
matrix_co.shape

(189559, 3990)

In [64]:
arr_cxd[2]

array([ 7.])

In [65]:
#80/20, 80 for 10-fold cross validation, 20 for test set holdout
create_seed(0)
indices = np.random.permutation(matrix_co.shape[0])
tsize = int(matrix_co.shape[0]*.8)

In [66]:
# customer to order matrix
matrix_co_training_idx, matrix_co_testing_idx  = indices[:tsize], indices[tsize:]
matrix_co_training, matrix_co_testing = matrix_co[matrix_co_training_idx,:], matrix_co[matrix_co_testing_idx,:]

In [67]:
# demo matrix
arr_cxd_training_idx, arr_cxd_testing_idx = indices[:tsize], indices[tsize:]
arr_cxd_training, arr_cxd_testing = arr_cxd[arr_cxd_training_idx,:], arr_cxd[arr_cxd_testing_idx,:]

In [68]:
matrix_co_training.shape, arr_cxd_training.shape, matrix_co.shape, arr_cxd.shape

((151647, 3990), (151647, 1), (189559, 3990), (189559, 1))

In [69]:
# put matrixes in to get cross validation recall
# generate recommendations
# mco = matrix of customer to order [customer x items]
# num_rec = number of recommendations
# num_folds = k number of folds for cross validation
# recall_remove = removed number from purchase history
def collab_recall_validation_tester(mco,num_rec,num_folds,recall_remove):
    # cross validation n-folds
    start_time = time.time()
    kf = KFold(n_splits=num_folds)
    list_total_acc=[]
    k_index = 0
    
    list_gen_acc = []
    
    for train, test in kf.split(mco):
        print("start new k={0},train set={1},validation set={2}".format(k_index,mco[train].shape[0],mco[test].shape[0]))
        
        list_sub_acc = []
        # build co-occurrence matrix
        coo_matrix = create_cocmatrix(mco[train])

        # loop through test set
        # for each customer in test pool
        for i in range(mco[test].shape[0]):      
            p_vec = mco[test][i,:]
            list_original_purchases = np.where(p_vec > 0)[0]
            # only run tests on customers with > recall_remove purchases for prediction
            if len(list_original_purchases) > recall_remove:
                # randomly select indexes to leave out
                list_removed_purchases = random.sample(list(list_original_purchases),recall_remove)
                # remove
                list_modified_purchases = list(set(list_original_purchases) - set(list_removed_purchases))
                # get sum all purchases except ones left out
                list_summed_coo_vec = gen_recom(list_modified_purchases,coo_matrix,num_rec)
                # check if return recommendations are in list
                list_recommended_match = set(list_removed_purchases) & set(list_summed_coo_vec)
                acc = len(list_recommended_match)/float(len(list_removed_purchases))
                list_sub_acc.append(acc)
                
        mean_sub_acc = np.mean(list_sub_acc)
        print ("** number of elements in calculation {0}".format(len(list_sub_acc)))
        #print list_sub_acc
        print ("** average fold accuracy {0}".format(mean_sub_acc))
        list_total_acc.append(mean_sub_acc)
        k_index += 1
        print ("**time elapsed {0}".format((time.time() - start_time)))
        print ("------------------------------------end")
        
    print ("list of total accuracy for each fold {0}".format(list_total_acc))
    average = np.mean(list_total_acc)
    print ("{0}-fold total average {1}".format(num_folds,average))
    print ("time elapsed {0}".format((time.time() - start_time)))
    return average

In [70]:
# put matrixes in to get cross validation coverage
# generate recommendations
# mco = matrix of customer to order [customer x items]
# num_rec = number of recommendations
# num_folds = k number of folds for cross validation
# recall_remove = removed number from purchase history
# list_catid = category customer is interested in
def collab_catalog_validation_tester(mco,num_rec,num_folds,recall_remove):
    # cross validation n-folds
    start_time = time.time()
    kf = KFold(n_splits=num_folds)
    list_total_coverage=[]
    k_index = 0
    
    for train, test in kf.split(mco):
        set_products = set()
        # build co-occurrence matrix
        print("start new k={0},train set={1},validation set={2}".format(k_index,mco[train].shape[0],mco[test].shape[0]))
        coo_matrix = create_cocmatrix(mco[train])

        # loop through test set
        # for each customer in test pool
        for i in range(mco[test].shape[0]):
            # purchase vector
            p_vec = mco[test][i,:]
            list_original_purchases = np.where(p_vec > 0)[0]
            # get sum all purchases
            list_summed_coo_vec = gen_recom(list_original_purchases,coo_matrix,num_rec)    
            set_products = set_products | set(list_summed_coo_vec)
                        
        mean_sub_cov = float(len(set_products))/p_vec.shape[0]
        print ("**length fold set {0}".format(float(len(set_products))))
        print ("**average fold coverage {0}".format(mean_sub_cov))
        list_total_coverage.append(mean_sub_cov)
        k_index += 1
        print ("**time elapsed {0}".format((time.time() - start_time)))
        print ("------------------------------------end")
    
    print ("list of total coverage for each fold {0}".format(list_total_coverage))
    average = np.mean(list_total_coverage)
    print ("{0}-fold total average {1}".format(num_folds,average))
    print ("time elapsed {0}".format((time.time() - start_time)))
    return average

In [79]:
def demo_filter(mco,mdo,clusters,maximum_rows,minimum_rows,num_rec,num_folds,recall_remove):
    accuracy = []
    coverage = []
    
    for c in clusters:
        cidx = get_indices(mdo, 0, c) # look for cluster in col 0
        print (len(cidx))
        if len(cidx) == 0:
            print ("cluster: {0} not found".format(c))
            continue

        if len(cidx) < minimum_rows:
            print ("too little data with cluster {0} found".format(c))
            continue

        cidx = check_idx_size(cidx,maximum_rows) # limit size
        print ("\ncluster: {0}, size: {1}, {2}".format(c, len(cidx), len(mco)))
                             
        total_accuracy = collab_recall_validation_tester(mco[cidx],num_rec,num_folds,recall_remove)
        accuracy.append((c,mco[cidx].shape[0],total_accuracy))
        total_coverage = collab_catalog_validation_tester(mco[cidx],num_rec,num_folds,recall_remove)
        coverage.append((c,mco[cidx].shape[0],total_coverage))
        
    return accuracy, coverage

In [80]:
seed = 0
create_seed(seed)
maximum_rows = 25000
minimum_rows = 1000

demo_accuracy, demo_coverage = demo_filter(matrix_co_training,arr_cxd_training,\
                                   unique_clusters,maximum_rows,minimum_rows,10,10,1)

3942

cluster: 0.0, size: 3942, 151647
start new k=0,train set=3547,validation set=395
** number of elements in calculation 120
** average fold accuracy 0.525
**time elapsed 2.25282621383667
------------------------------------end
start new k=1,train set=3547,validation set=395
** number of elements in calculation 118
** average fold accuracy 0.4322033898305085
**time elapsed 4.435358285903931
------------------------------------end
start new k=2,train set=3548,validation set=394
** number of elements in calculation 112
** average fold accuracy 0.5178571428571429
**time elapsed 6.64156699180603
------------------------------------end
start new k=3,train set=3548,validation set=394
** number of elements in calculation 124
** average fold accuracy 0.532258064516129
**time elapsed 9.027894020080566
------------------------------------end
start new k=4,train set=3548,validation set=394
** number of elements in calculation 113
** average fold accuracy 0.4424778761061947
**time elapsed 11.45

start new k=6,train set=22500,validation set=2500
**length fold set 742.0
**average fold coverage 0.18596491228070175
**time elapsed 545.2503578662872
------------------------------------end
start new k=7,train set=22500,validation set=2500
**length fold set 722.0
**average fold coverage 0.18095238095238095
**time elapsed 629.0567820072174
------------------------------------end
start new k=8,train set=22500,validation set=2500
**length fold set 740.0
**average fold coverage 0.18546365914786966
**time elapsed 708.9832320213318
------------------------------------end
start new k=9,train set=22500,validation set=2500
**length fold set 754.0
**average fold coverage 0.18897243107769424
**time elapsed 791.0139000415802
------------------------------------end
list of total coverage for each fold [0.17669172932330826, 0.17769423558897243, 0.17593984962406015, 0.18145363408521303, 0.18295739348370926, 0.18095238095238095, 0.18596491228070175, 0.18095238095238095, 0.18546365914786966, 0.1889724

start new k=1,train set=22500,validation set=2500
**length fold set 739.0
**average fold coverage 0.18521303258145363
**time elapsed 155.73076391220093
------------------------------------end
start new k=2,train set=22500,validation set=2500
**length fold set 757.0
**average fold coverage 0.18972431077694235
**time elapsed 229.96906685829163
------------------------------------end
start new k=3,train set=22500,validation set=2500
**length fold set 718.0
**average fold coverage 0.1799498746867168
**time elapsed 303.35137701034546
------------------------------------end
start new k=4,train set=22500,validation set=2500
**length fold set 728.0
**average fold coverage 0.1824561403508772
**time elapsed 379.6438949108124
------------------------------------end
start new k=5,train set=22500,validation set=2500
**length fold set 729.0
**average fold coverage 0.18270676691729323
**time elapsed 452.26781010627747
------------------------------------end
start new k=6,train set=22500,validation se

start new k=7,train set=12769,validation set=1419
** number of elements in calculation 358
** average fold accuracy 0.5810055865921788
**time elapsed 216.6797571182251
------------------------------------end
start new k=8,train set=12770,validation set=1418
** number of elements in calculation 349
** average fold accuracy 0.6303724928366762
**time elapsed 247.19447898864746
------------------------------------end
start new k=9,train set=12770,validation set=1418
** number of elements in calculation 372
** average fold accuracy 0.5564516129032258
**time elapsed 275.31871700286865
------------------------------------end
list of total accuracy for each fold [0.62894736842105259, 0.52956989247311825, 0.56111111111111112, 0.54838709677419351, 0.55371900826446285, 0.59114583333333337, 0.55408970976253302, 0.58100558659217882, 0.63037249283667618, 0.55645161290322576]
10-fold total average 0.5734799712471885
time elapsed 275.3197009563446
start new k=0,train set=12769,validation set=1419
**le

start new k=2,train set=22500,validation set=2500
** number of elements in calculation 700
** average fold accuracy 0.6271428571428571
**time elapsed 234.98635697364807
------------------------------------end
start new k=3,train set=22500,validation set=2500
** number of elements in calculation 673
** average fold accuracy 0.6017830609212481
**time elapsed 313.1446897983551
------------------------------------end
start new k=4,train set=22500,validation set=2500
** number of elements in calculation 667
** average fold accuracy 0.6221889055472264
**time elapsed 387.7703297138214
------------------------------------end
start new k=5,train set=22500,validation set=2500
** number of elements in calculation 655
** average fold accuracy 0.601526717557252
**time elapsed 467.0572168827057
------------------------------------end
start new k=6,train set=22500,validation set=2500
** number of elements in calculation 694
** average fold accuracy 0.600864553314121
**time elapsed 545.595764875412
--

In [81]:
demo_accuracy

[(0.0, 3942, 0.50009549728407132),
 (1.0, 25000, 0.61608126661938301),
 (2.0, 5353, 0.55112967472187258),
 (3.0, 25000, 0.6172684944364375),
 (4.0, 25000, 0.61552034509922204),
 (5.0, 14188, 0.57347997124718852),
 (6.0, 2273, 0.48378859805090724),
 (7.0, 25000, 0.60807781756713142)]

In [82]:
np.save('demo_clustering_cluster_accuracy1.npy',\
        [(maximum_rows,minimum_rows,seed,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))]+demo_accuracy)

In [83]:
demo_coverage

[(0.0, 3942, 0.093884711779448626),
 (1.0, 25000, 0.18170426065162903),
 (2.0, 5353, 0.08834586466165413),
 (3.0, 25000, 0.18140350877192984),
 (4.0, 25000, 0.18751879699248122),
 (5.0, 14188, 0.14057644110275688),
 (6.0, 2273, 0.048796992481203005),
 (7.0, 25000, 0.17290726817042607)]

In [84]:
np.save('demo_clustering_cluster_coverage1.npy',\
        [(maximum_rows,minimum_rows,seed,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))]+demo_coverage)

In [92]:
!pwd

/Users/jc/Documents/DSE/joh112/dse203-group-project_newer2/machine_learning/ValidationTests
